## Text Classification using MultinomialNB

In [ ]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
import numpy as np
import re
import os
import math
from sklearn.metrics import accuracy_score

In [ ]:
#stop word file location you can change the path as per your convinence
with open('C:/Users/priya/Downloads/20_newsgroups/stopwords.txt',encoding='UTF-8') as file:
    stop_words_txt=file.read()
    stop_words={}
    for st_word in stop_words_txt.splitlines():
        stop_words[st_word.lower()]=stop_words.get(st_word.lower(),0)+1

In [ ]:
# location of newsgroup you can change the path as per your convinence
rootdir=r"C:\Users\priya\Downloads\20_newsgroups\20_newsgroups"
word_freq={}
for folder in os.listdir(rootdir):
    new_rootdir=os.path.join(rootdir, folder)
    for each_subfile in os.listdir(new_rootdir)[:50]:
        file_=os.path.join(new_rootdir,each_subfile)
        with open(file_) as file:
            #read each file and add words to word_freq dictionary
            file_txt=file.read()
            file_txt=file_txt.lower()
            # ignored text till the occurance of line: to filter the data
            file_txt=file_txt.partition('lines:')[2]
            for line in file_txt.splitlines():
                for word in re.split(':|;| |,|\n|\/|\.|\t|!|-',line):
                    word=re.sub("[^a-zA-Z']", '', word)
                    word=word.lower()
                    if word.lower() in stop_words or word=='':
                        pass
                    else:
                        word_freq[word]=word_freq.get(word,0)+1
            file.close()

In [ ]:
word_freq=dict(sorted(word_freq.items(), key=lambda item: item[1],reverse=True))

In [ ]:
## get top freq k words from word_freq and add then as df column
k=3000
column_names=list(word_freq.keys())[:k]
column_names.append("_output_")
df=pd.DataFrame(columns=column_names)

In [ ]:
def make_dataframe():
    
    #built dataframe corresponding to most frequnt words
    rootdir=r"C:\Users\priya\Downloads\20_newsgroups\20_newsgroups"
    word_freq={}
    for folder in os.listdir(rootdir):
        new_rootdir=os.path.join(rootdir, folder)
        for each_subfile in os.listdir(new_rootdir):
            file_=os.path.join(new_rootdir,each_subfile)
            with open(file_) as file:
                #read each file and add words to dictionary
                file_txt=file.read()
                row_list=[0 for i in range(k)]
                for line in file_txt.splitlines():
                    for word in re.split(':|;| |,|\n|\/|\.|\t|!|-',line):
                        word=re.sub("[^a-zA-Z']", '', word)
                        word=word.lower()
                        if word in column_names:
                            row_list[column_names.index(word)]=row_list[column_names.index(word)]+1
                row_list.append(folder)
                df.loc[len(df.index)] = row_list
                file.close()
    return df

In [ ]:
input_data=make_dataframe()
input_data["_output_"]=y

In [ ]:
data=input_data
y=data["_output_"]
del data["_output_"]
x=data

In [ ]:
x_tarin,x_test,y_tain,y_test=train_test_split(x,y)

In [ ]:
clf=MultinomialNB()

In [ ]:
clf.fit(x_tarin,y_tain)

In [ ]:
predict=clf.predict(x_test)

In [ ]:
mat=confusion_matrix(y_test,predict)

In [ ]:
clf.score(x_test,y_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#data representation using heatmap
sns.heatmap(mat.T, square = True, annot=True, fmt = "d", xticklabels=list(set(y)),yticklabels=list(set(y)))
plt.xlabel("true labels")
plt.ylabel("predicted label")
plt.show()
# print("The accuracy is {}".format(accuracy_score(test_data.target, predicted_categories)))

## Custom implementation of Text Classifier

In [ ]:
def create_dictionary(train,column_names):
    unique_classes=set(train["_output_"])
    unique_features=column_names
    unique_features.pop()
    dict_class={}
    for class_ in unique_classes: 

        # build a dictionary of features with respect to each class
        data_corresponding_class=train[train["_output_"]==class_]        
        dict_class[class_]={}
    #     print(data_corresponding_class)
        del data_corresponding_class["_output_"]
    #     print(data_corresponding_class)
        total_sum=data_corresponding_class.sum().sum()
    #     print(total_sum)
        for feature in unique_features:
            dict_class[class_][feature]=calculate_probablity(data_corresponding_class[feature].sum(),total_sum) # calculate probablity
    return dict_class
            

In [ ]:
# calculating probablity for each feature
def calculate_probablity(favourable,total_outcomes):
    prob=math.log((favourable+1)/(total_outcomes+3000))
    return prob

In [ ]:
#creating dictionary of feature probablity with respecct to the calss
dict_=create_dictionary(input_data,column_names)

In [ ]:
#predict the output for the given test cases
def predictclass(dict_class,test_data,unique_classes):
#     del test_data["_output_"]
    predicted_output=[]
    for ele in test_data:
#         np_ele=np.array(ele)
        max_prob=-9999999
        class_to_be_selected=""
        pd_class_dict=pd.DataFrame.from_dict(dict_class)
        for each_class in unique_classes:
            a=np.array(pd_class_dict[each_class])
#             print(a)
            prob=((a)*(ele)).sum()
            if max_prob<prob:
                max_prob=prob
                class_to_be_selected=each_class
        predicted_output.append(class_to_be_selected)
    return predicted_output

In [ ]:
#converting dataframe to numpy array
p=x_test
d=p.to_numpy()

In [ ]:
#predicted output using custom Text Classification
out=predictclass(dict_,d,set(y))

In [ ]:
accuracy_score(y_test,out)

In [ ]:
#Creating confusion matrix 
m=confusion_matrix(y_test,out)

In [ ]:
#Drawing a heatmap for 
sns.heatmap(m.T, square = True, annot=True, fmt = "d", xticklabels=list(set(y)),yticklabels=list(set(y)))
plt.xlabel("true labels")
plt.ylabel("predicted label")
plt.show()